In [1]:
# Asynchronous
import nest_asyncio
nest_asyncio.apply()
import asyncio
import aiohttp

import tqdm
import requests
from pprint import pprint
import datetime
from bs4 import BeautifulSoup

import pandas as pd
import numpy as np
import json

import re
cookies = {
    'ccsid': '027-9406763-1350835',
    '__qca': 'P0-867993267-1671835417644',
    'p': 'BFnM5Q76wF0shLrczPEWD6Ai6glvalBOI0HQnM7wAheYffUa',
    'likely_has_account': 'true',
    'srb_8': '0_ar',
    'locale': 'en',
    'csm-sid': '438-4025740-7454350',
    'allow_behavioral_targeting': 'true',
    'session-id': '147-4067731-5921029',
    'lc-main': 'en_US',
    'logged_out_browsing_page_count': '2',
    'ubid-main': '131-6414756-0108314',
    'csm-hit': 'tb:9FKSJEW3PZPAQZKSV74Z+s-9FKSJEW3PZPAQZKSV74Z|1675549894918&t:1675549894919&adb:adblk_yes',
    'session-id-time': '2306269930l',
    'session-token': 'fmlMtndkzGiWYItedsruMqFs9+7dbrKkjWxwS050IWs57BCY8RQJGXxU8qPn+6lNucog+VtoV/qrLn6eBpe8GwFqeSA75LzytrwlvgKScLWr62XYbJJg36UcoVnTCNaBAUkT+jhZdWNLpHXEQ/T3fsq03ctsMB7GChLcVrF/10rEk0ETQ5LmuUN5ordUvBjgbgf/wLazQPbG5Ia8rmhh/Gyptwz9N0ilyQSLx5h7pVc1fl4z+U3O3seFJdO2xI8M',
    'x-main': '"QKUGsTWzgOIn1MQ?MLW6M7?W9sXe6vT8lOfYIRdLo30O9fDyRRuxqM6Up6QliSso"',
    'at-main': 'Atza|IwEBIE9-j0Q56LUPug78YuRWyKfqUI-71kqBUKBSv9o0pgXoEdWRTp3QanY2PVmzS5kuRKhJ-2Qf2Y2Xza8aOzNTehTteyzG_d1HL64CDYGBvcFiM3J3K6q-_iEKTHPqIuFGTC7GBQffrkFancRYWCUTNK7ii9qpL1Qvc-lEAmj5xJFsos2I98msXxsTvgVFMUfr1rhJK6CfvkpCAIziz2lPcltlQCWaFsR10_Le_ytyKVYeIZ18m-F1bXhhejn7mTwQJl0',
    'sess-at-main': '"f/c/JR/xicYEmsJ7sGnJySc4uAQ6ojXuaOFFChXMGho="',
    '_session_id2': '64c06a0ae25fee31093af8a3bf3f79e1',
}

headers = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,/;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'Accept-Language': 'en-US,en;q=0.9',
    'Cache-Control': 'max-age=0',
    'Connection': 'keep-alive',
    # 'Cookie': 'ccsid=027-9406763-1350835; __qca=P0-867993267-1671835417644; p=BFnM5Q76wF0shLrczPEWD6Ai6glvalBOI0HQnM7wAheYffUa; likely_has_account=true; srb_8=0_ar; locale=en; csm-sid=438-4025740-7454350; allow_behavioral_targeting=true; session-id=147-4067731-5921029; lc-main=en_US; logged_out_browsing_page_count=2; ubid-main=131-6414756-0108314; csm-hit=tb:9FKSJEW3PZPAQZKSV74Z+s-9FKSJEW3PZPAQZKSV74Z|1675549894918&t:1675549894919&adb:adblk_yes; session-id-time=2306269930l; session-token=fmlMtndkzGiWYItedsruMqFs9+7dbrKkjWxwS050IWs57BCY8RQJGXxU8qPn+6lNucog+VtoV/qrLn6eBpe8GwFqeSA75LzytrwlvgKScLWr62XYbJJg36UcoVnTCNaBAUkT+jhZdWNLpHXEQ/T3fsq03ctsMB7GChLcVrF/10rEk0ETQ5LmuUN5ordUvBjgbgf/wLazQPbG5Ia8rmhh/Gyptwz9N0ilyQSLx5h7pVc1fl4z+U3O3seFJdO2xI8M; x-main="QKUGsTWzgOIn1MQ?MLW6M7?W9sXe6vT8lOfYIRdLo30O9fDyRRuxqM6Up6QliSso"; at-main=Atza|IwEBIE9-j0Q56LUPug78YuRWyKfqUI-71kqBUKBSv9o0pgXoEdWRTp3QanY2PVmzS5kuRKhJ-2Qf2Y2Xza8aOzNTehTteyzG_d1HL64CDYGBvcFiM3J3K6q-_iEKTHPqIuFGTC7GBQffrkFancRYWCUTNK7ii9qpL1Qvc-lEAmj5xJFsos2I98msXxsTvgVFMUfr1rhJK6CfvkpCAIziz2lPcltlQCWaFsR10_Le_ytyKVYeIZ18m-F1bXhhejn7mTwQJl0; sess-at-main="f/c/JR/xicYEmsJ7sGnJySc4uAQ6ojXuaOFFChXMGho="; _session_id2=64c06a0ae25fee31093af8a3bf3f79e1',
    'If-None-Match': 'W/"087d62f3d9fcbed8836cb66e1e0c2bef"',
    'Referer': 'https://www.goodreads.com/',
    'Sec-Fetch-Dest': 'document',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-User': '?1',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
    'sec-ch-ua': '"Not_A Brand";v="99", "Google Chrome";v="109", "Chromium";v="109"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
}

params = {
    'ref': 'nav_comm_people',
}


In [3]:

user_types = ['https://www.goodreads.com/user/best_reviewers', 'https://www.goodreads.com/user/top_reviewers','https://www.goodreads.com/user/top_readers']
countries = ['?country=CA','?country=all','?country=US','?country=US','?country=AU','?country=SC', '?country=NZ','?country=IE','?country=GB', '?country=SG', '?country=GB','?country=DO', '?country=TT', '?country=MT', '?country=BB','?country=LC','?country=GY']
time_frame = ['&duration=w','&duration=m', '&duration=y','&duration=a']

users = set()
for type in tqdm.tqdm(user_types): 
    for country in tqdm.tqdm(countries):
        for time in tqdm.tqdm(time_frame):
            url = type+country+time
            response = requests.get(url, params=params, cookies=cookies, headers=headers)
            soup = BeautifulSoup(response.content, 'html.parser')
            tab = soup.find("table",{"class":"tableList"})
            if tab is None:
                continue
            else:
                trs = tab.find_all('tr')
                for t in trs: 
                    s = t.find_all('td')[1].find('a',href = True)['href']
                    user_id = re.findall(r'\d+',s)[0]
                    users.add(user_id)

with open('users.json', 'w') as file:
    json.dump(list(users), file, indent=2)

100%|██████████| 3/3 [05:44<00:00, 114.80s/it]

7811


In [14]:
# THIS MODULE GENERATES A USER PAGES TO ITERATE OVER
# each page contains a list of 20 books that user has read

async def get_user_review_pages(session, user):
    async with session.get("https://www.goodreads.com/review/list/"+user,
                                 params=params, cookies=cookies, headers=headers) as response:
        ur_page = await response.text()
        ur_soup = BeautifulSoup(ur_page, 'html.parser')

        # find number of review pages
        if ur_soup.find(id='reviewPagination') == None:
            return(user)

        
        review_pages = int(ur_soup.find(id='reviewPagination').find_all("a")[-2].text)
        if review_pages > 5:
            check_pages = np.random.choice(review_pages, replace=False, size=5)
        else: 
            check_pages = [i for i in range(review_pages)]

        new_urls = []
        for p in check_pages:
            new_urls.append((f"https://www.goodreads.com/review/list/{user}?page={p+1}",user))
        review_urls.extend(new_urls)
        
        # Finalize
        print("READ", end=' ')
        return("")

async def download_comments():
    async with aiohttp.ClientSession(timeout = aiohttp.ClientTimeout(total=60*60*24)) as session:

        tasks = []
        for user in users:
            tasks.append(asyncio.ensure_future(get_user_review_pages(session, user)))

        failed_users = await asyncio.gather(*tasks)
        failed_users_res = []
        for f_user in failed_users:
            if f_user != "":
                failed_users_res.append(f_user)

        return(failed_users_res)

review_urls = []
failed_users = asyncio.run(download_comments())

with open('reviews_urls.json', 'w') as file:
    json.dump(review_urls, file, indent=2)

READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ 

In [18]:
print(failed_users)
print(len(review_urls))

[]
490


In [64]:
# THIS MODULE ITERATES OVER ALL USER DATA
# each page contains a list of 20 books that user has read
# We then save each book to the list of books to gather data for

with open('reviews_urls.json', 'r') as file:
    review_urls = json.load(file)


# FIGURE OUT TIMEOUT ERRORS
async def get_user_read_books(session, url, user):
    async with session.get(url, params=params, cookies=cookies, headers=headers) as response:
        ur_page = await response.text()
        ur_soup = BeautifulSoup(ur_page, 'html.parser')

        # Find what each user has read (as book url)
        user_read = []
        books = ur_soup.find(id='booksBody')

        # ERROR
        if books == None:
            return ([url, user])

        # SAVE
        books = books.find_all('tr')
        for b in books: 
            book_link = b.find('a',href = True)['href']
            user_read.append((book_link,user))
        read_history.extend(user_read)

        # Finalize
        print(f"READ", end=' ')
        return([])
        # return(user_read, None)

async def get_read_books(urls_to_view):
    async with aiohttp.ClientSession(timeout=aiohttp.ClientTimeout(total=60*60*24, sock_connect=10)) as session:

        tasks = []
        for rev_url, user in urls_to_view:
            tasks.append(asyncio.ensure_future(get_user_read_books(session, rev_url, user)))

        failed_urls = await asyncio.gather(*tasks)
        
        failed_urls_return = []
        for url in failed_urls:
            if url != []:
                failed_urls_return.append(url)

        return(failed_urls_return)


read_history = []
while len(review_urls) > 0:
    review_urls = asyncio.run(get_read_books(review_urls))
    print(review_urls)

READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ 

In [53]:
len(read_history)

69062

In [ ]:
# len(read_history)

stringy = "READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ READ"
len(stringy.split(" "))

1096

In [8]:
read_history

[('/book/show/63852686-hey-presto', '31134832'),
 ('/book/show/52263002-love-is-for-later', '31134832'),
 ('/book/show/53505366-project-crush', '31134832'),
 ('/book/show/1753489.The_Wild_One', '31134832'),
 ('/book/show/61688520-for-her-consideration', '31134832'),
 ('/book/show/63189063-the-woman-i-found-in-me', '31134832'),
 ('/book/show/75434380-chaos-agent', '31134832'),
 ('/book/show/19197706-wild-things', '31134832'),
 ('/book/show/50854480-on-second-thought', '31134832'),
 ('/book/show/60559781-encore-performance', '31134832'),
 ('/book/show/58751887-curtain-call', '31134832'),
 ('/book/show/61303853-royal-bite', '31134832'),
 ('/book/show/75428323-her-rival-dragon-mate', '31134832'),
 ('/book/show/75434345-the-fixer', '31134832'),
 ('/book/show/62854145-undead-at-large', '31134832'),
 ('/book/show/63046198-red-moon-rising', '31134832'),
 ('/book/show/75419130-hearts-in-the-wind', '31134832'),
 ('/book/show/75444188-last-new-beginning', '31134832'),
 ('/book/show/62824934-the-h

In [ ]:
user_ID_1 = []
book_description_1 = []
book_author_1 = []
book_title_1 = []
book_rating_1 = []
book_pages_1 = []
num_of_rating_1 = []
num_of_review_1 = []
genre_1 = []
publish_1 = []
language_1 = []
award_1 = []
isbn_1 = []

for user in tqdm(users):
    links=[]
    for num in [1,2,3]:
        response1 = requests.get("https://www.goodreads.com/review/list/"+user+"?page="+str(num),
                                 params=params, cookies=cookies, headers=headers)
        soup = BeautifulSoup(response1.content, 'html.parser')

        tab_book = soup.find(id_='booksBody')
        tab_book_1 = tab_book.find_all('tr')

        for t in tab_book_1: 
            s = t.find('a',href = True)['href']
            links.append(s)
        
    full_url = [urljoin(base_site, relativeurl) for relativeurl in links]
    book_url = full_url
    # book_url = [url for url in full_url if "/book/show" in url]
    
    # while 'https://www.goodreads.com/book/show/63337133-rouge' in book_url: 
    #     book_url.remove('https://www.goodreads.com/book/show/63337133-rouge')
    # while 'https://www.goodreads.com/book/show/63141623-house-woman' in book_url: 
    #     book_url.remove('https://www.goodreads.com/book/show/63141623-house-woman')
    
    user_ID = []
    book_description = []
    book_author = []
    book_title = []
    book_rating = []
    book_pages = []
    num_of_rating = []
    num_of_review = []
    genre = []
    publish = []
    language = []
    award = []
    isbn = []

    import re
    import json

    faulty=[]

    for url in book_url:
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'}
        note_resp = requests.get(url, headers=headers,params=params, cookies=cookies)

        #if note_resp.status_code == 200:
         #   print("URL{}: {}".format(i+1, url))

        #else:
         #   print('Status code{}: Skipping URL #{}: {}'.format(note_resp.status_code, i+1, url))
          #  i = i+1
           # continue

        try:
            #get HTML from url page
            note_html = note_resp.content

            #create beautifulsoup object for url page
            note_soup = BeautifulSoup(note_html, 'html.parser')

            #Extract Author particulars


            pages_divs = note_soup.find_all("p", {"data-testid": "pagesFormat"})
            pages_text = pages_divs[0].text


            num_of_rating_divs = note_soup.find_all("span", {"data-testid": "ratingsCount"})
            num_of_rating_text = num_of_rating_divs[0].text


            num_of_review_divs = note_soup.find_all("span", {"data-testid": "reviewsCount"})
            num_of_review_text = num_of_review_divs[0].text


            genre_divs = note_soup.find_all("a", {"class": "Button Button--tag-inline Button--small"})
            genre_text = genre_divs[1].text


            publish_divs = note_soup.find_all("p", {"data-testid":"publicationInfo"})
            publish_text = publish_divs[0].text

            author_divs = note_soup.find_all("span",{"class":"ContributorLink__name"})
            author_text = author_divs[0].text



            title_divs = note_soup.find_all("h1", {"class": "Text Text__title1"})
            title_text = title_divs[0].text


            rating_divs = note_soup.find_all("div", {"class": "RatingStatistics__rating"})
            rating_text = rating_divs[0].text


            json_str = note_soup.find_all("script",{'type':'application/ld+json'})[0].string
            data = json.loads(json_str)


            if 'awards' in data.keys():
                award.append(1)
            else :
                award.append(0)

            if 'isbn' in data.keys():
                isbn.append(data.get("isbn"))
            else :
                isbn.append('NA')


            description_divs = note_soup.find_all("span", {"class": "Formatted"})
            try:
                description_text = description_divs[0].text
            except IndexError:
                description_text = "Nil"


            book_author.append(author_text)
            book_description.append(description_text)
            book_title.append(title_text)
            book_rating.append(float(rating_text))
            book_pages.append(float(re.search(r'\d+', pages_text.replace(',', '')).group()))
            num_of_rating.append(float(re.search(r'\d+', num_of_rating_text.replace(',', '')).group()))
            num_of_review.append(float(re.search(r'\d+', num_of_review_text.replace(',', '')).group()))
            genre.append(genre_text)
            publish.append(publish_text.replace("First published ", ""))
            language.append(data.get("inLanguage"))
            user_ID.append(user)

        except :
            faulty.append(url)


    book_description = book_description[:len(genre)]
    book_author = book_author[:len(genre)]
    book_title = book_title[:len(genre)]
    book_rating = book_rating[:len(genre)]
    book_pages = book_pages[:len(genre)]
    num_of_rating = num_of_rating[:len(genre)]
    num_of_review = num_of_review[:len(genre)]
    genre = genre[:len(genre)]
    publish = publish[:len(genre)]
    language = language[:len(genre)]
    award = award[:len(genre)]
    isbn = isbn[:len(genre)]
    user_ID = user_ID[:len(genre)]
    
    user_ID_1.extend(user_ID)
    book_description_1.extend(book_description)
    book_author_1.extend(book_author)
    book_title_1.extend(book_title)
    book_rating_1.extend(book_rating)
    book_pages_1.extend(book_pages)
    num_of_rating_1.extend(num_of_rating)
    num_of_review_1.extend(num_of_review)
    genre_1.extend(genre)
    publish_1.extend(publish)
    language_1.extend(language)
    award_1.extend(award)
    isbn_1.extend(isbn)

    

In [264]:
final=pd.DataFrame({'Description':book_description_1,
    'Author':book_author_1,
    'Title':book_title_1,
    'Rating':book_rating_1,
    'Pages':book_pages_1,
    'Num of Ratings':num_of_rating_1,
    'Num of Review':num_of_review_1,
    'Genre':genre_1,
    'Publish Date':publish_1,
    'Language':language_1,
    'Award':award_1,
    'ISBN':isbn_1,
    'USERID':user_ID_1})

In [266]:
final.head(50)

,Description,Author,Title,Rating,Pages,Num of Ratings,Num of Review,Genre,Publish Date,Language,Award,ISBN,USERID
0,Wonder Woman isn’t exactly thrilled to be batt...,Cecil Castellucci,Sensation Comics Featuring Wonder Woman (2014-...,4.00,23.0,13.0,0.0,Wonder Woman,"February 12, 2015",English,0,NA,22106879
1,“Girls’ Day Out” part 1 of 2! Lois Lane doesn’...,Cecil Castellucci,Sensation Comics Featuring Wonder Woman (2014-...,4.00,23.0,13.0,0.0,Wonder Woman,"February 5, 2015",English,0,NA,22106879
2,Wonder Woman’s latest mission is to help India...,Heather Nuhfer,Sensation Comics Featuring Wonder Woman (2014-...,3.73,23.0,15.0,2.0,Wonder Woman,"January 29, 2015",English,0,NA,22106879
3,Diana’s adventure in Man’s World takes an unex...,James Tynion IV,Sensation Comics Featuring Wonder Woman (2014-...,4.11,23.0,27.0,3.0,Wonder Woman,"January 22, 2015",English,0,NA,22106879
4,Wonder Woman has been summoned to the Isle of ...,Lauren Beukes,Sensation Comics Featuring Wonder Woman (2014-...,3.60,23.0,20.0,3.0,Wonder Woman,"January 8, 2015",English,0,NA,22106879
5,“Wonder World” part 1 of 2! Teenaged Diana com...,James Tynion IV,Sensation Comics Featuring Wonder Woman (2014-...,4.36,23.0,22.0,3.0,Wonder Woman,"January 15, 2015",English,0,NA,22106879
6,The atmosphere of Venus is hostile to human (a...,Alex de Campi,Sensation Comics Featuring Wonder Woman (2014-...,4.06,23.0,16.0,1.0,Wonder Woman,"January 1, 2015",English,0,NA,22106879
7,"""Venus Rising"" part 1 of 2. Diana agreed to re...",Alex de Campi,Sensation Comics Featuring Wonder Woman (2014-...,4.00,23.0,17.0,2.0,Wonder Woman,"December 25, 2014",English,0,NA,22106879
8,A combat pilot gets a little help from Wonder ...,Amy Chu,Sensation Comics Featuring Wonder Woman (2014-...,3.88,23.0,16.0,1.0,Wonder Woman,"December 18, 2014",English,0,NA,22106879
9,SERIES PREMIERETwo obsessed NYPD detectives hu...,Peter J. Tomasi,Blood Tree #1,4.00,32.0,2.0,1.0,author-american,"February 1, 2023",English,0,NA,22106879
